<a href="https://colab.research.google.com/github/kidd000/Acerbi_Textbook/blob/main/acerbi_CH4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

第3章では、ある文化的形質が他の形質よりも本質的にコピーされやすい場合について見てきました。ここでは、形質が同等であるにもかかわらず、個体は集団の特性、特に他の個体がすでにその形質を持っている場合に、ある形質を採用する可能性が高くなるという、他の種類の伝達バイアスについて見ていきます。先述したように、これらはしばしば「間接的」または「文脈」バイアスと呼ばれます）。


# **4.1   　同調伝達の論理**


第一の可能性として、集団内での形質の頻度、すなわち、他の個体がどれだけの数の形質をすでに持っているかに影響される可能性があります。同調（または「正の頻度依存バイアス」）が最も研究されています。ここでは、個体は、その固有の特徴に関係なく、集団の中で最も一般的な形質を採用する可能性が不釣り合いなほど高くなる。反対に、最も一般的でない形質がコピーされる可能性が高い場合、反同調または負の頻度依存バイアスも考えられる。これは実際の生活ではおそらくあまり一般的ではないが)。

例えば、母集団の中で形質Aの頻度が0.7で、残りの人が形質Bを持っているとするとします。偏りのない学習者は，0.7と全く等しい確率で形質Aを採用するだろう。これは偏りのない伝達であり，（第1章：前世代のメンバーをランダムに選ぶことであり、採用の確率は前世代の中でのその形質の頻度に等しい）で説明したモデルで起こることです。

一方、同調主義的な学習者は、0.7よりも大きな確率で形質Aを採用します。言い換えれば、普及している形質は、偏りのない伝達に比べて「採用ブースト」を受けることになります。普及していない形質は、それに相当する「採用ペナルティ」を受けます。このブーストやペナルティの大きさは、パラメータによって制御することができます。

我々のモデルでは、シンプルにしておきましょう。個人が母集団全体からサンプリングすると仮定するのではなく（大規模な母集団ではありえない）、3人だけのデモンストレータを無作為に選ぶと仮定してみましょう。なぜ3人なのか？これは、多数派（すなわち、2対1）を生み出すことができるデデモンストレータの最小数であり、同調を実装するために必要な数です。2人のデモンストレータが1つの形質を持っていて、もう1人のデモンストレータが別の形質を持っている場合、多数派の形質が採用される確率を高め、少数派の形質が採用される確率を下げたい。

（中略）

これらの前提条件を、これまで構築してきたような個人ベースのモデルに実装してみましょう。前述のように、N個の個体がそれぞれAかBの2つの形質のいずれかを持っていると仮定します。世代t =1におけるAの初期頻度はp0です。関数に直行するのではなく、一歩一歩進んでみましょう。

最初に、パラメータ N と p0 を以前と同様に指定し、さらに新しい同調パラメータ D を指定します。また、通常の母集団を作成し、p0 で指定された割合で A と B で満たします。





In [ ]:
import random
import numpy as np
import pandas as pd

N = 100
p_0 = 0.5
D = 1

# 乱数種を設定
np.random.seed(1)

# 文化形質を設定
cultural_trait = ["A","B"]

# 1世代目のpopulationを作る
population = list(np.random.choice(cultural_trait, size = N, p = [p_0, 1-p_0]))

# 1世代目のpopulationの中身を見てみる
population

'B'

ここで、次の世代の新しい個体ごとに、現在の個体群から無作為に3人のデモンストレータを選ぶ、デモンストレータと呼ばれる別のデータフレームを作成します。したがって、3つの列/変数（それぞれのデモンストレータに1つずつ）、N個の行（各個人に1つずつ）が必要となります。各列をpopulationデータフレームからランダムに選ばれた形質で埋めます。

In [ ]:
# デモンストレータのデータフレームを作る-1世代目のpopulationからランダムに3人ずつ選ぶ試行をN人分
demonstrators = pd.DataFrame(np.arange(N*3).reshape(N,3)).applymap(lambda x: np.random.choice(population))

# デモンストレータのデータフレームの中身を見てみる
demonstrators

,0,1,2
0,B,B,A
1,A,A,B
2,B,A,B
3,A,B,A
4,B,B,B
...,...,...,...
95,B,A,A
96,B,A,B
97,B,B,B
98,B,B,B


ここでは、各行が、それぞれの新しい次世代の個人によってランダムに選ばれた３つのデモンストレーターの文化形質が表されていると考えてください。ここでは、これら３人のデモンストレーターの文化形質の組み合わせについて、Ａを採用する確率を計算したいと思います。

まず、各組み合わせに含まれるAの数を求める。そして、表1の確率に基づいて母集団の形質を入れ替えます。3人中2人がAを持っている場合は確率2/3+D/3でAとし、そうでない場合はBとする。3人のうち1人がAを持っている場合は、1/3-D/3の確率でAとし、そうでない場合はBとする。



In [ ]:
print("表1 3人のデモンストレータが持つ文化形質のそれぞれの組み合わせについて、形質Aを採用する確率")
Table = pd.DataFrame({"Demonstrator1":["A","A","A","B","A","B","B","B"], "Demonstrator2":["A","A","B","A","B","A","B","B"],"Demonstrator3":["A","B","A","A","B","B","A","B"], "Probability of adopting trait A":["1", "2/3+D/3","2/3+D/3","2/3+D/3","1/3-D/3","1/3-D/3","1/3-D/3","0"]})
Table

表1 3人のデモンストレータが持つ文化形質のそれぞれの組み合わせについて、形質Aを採用する確率


,Demonstrator1,Demonstrator2,Demonstrator3,Probability of adopting trait A
0,A,A,A,1
1,A,A,B,2/3+D/3
2,A,B,A,2/3+D/3
3,B,A,A,2/3+D/3
4,A,B,B,1/3-D/3
5,B,A,B,1/3-D/3
6,B,B,A,1/3-D/3
7,B,B,B,0
